# Finding optimal adjustment sets

This notebook illustrates the use of the algorithms developed in [Smucler, Sapienza and Rotnitzky (2021)](https://doi.org/10.1093/biomet/asab018) and [Smucler and Rotnitzky (2022)](https://www.degruyter.com/document/doi/10.1515/jci-2022-0015/html) to compute efficient backdoor sets under various constraints.

### Preliminaries

In [ ]:
from dowhy.causal_graph import CausalGraph
from dowhy.causal_identifier import CausalIdentifier

Consider the design of the following hypothetical observational study discussed in [Shrier & Platt (2008)](https://doi.org/10.1186/1471-2288-8-70). The aim of the study is to assess the
effect of warm-up exercise on injury after playing sports. Suppose that a researcher postulates
that the graph below represents a causal graphical model. The vertex warm-up is the treatment variable, which stands for the type of exercise an athlete performs prior to playing sports,
and the vertex injury stands for the outcome variable. Suppose that the goal is to estimate and
compare the interventional means corresponding to different individualised treatment rules. Each
rule prescribes the type of warm-up exercise as a function of previous injury and team motivation. Suppose that due to practical limitations, the variables genetics, pre-grame proprioception,
intra-game proprioception and tissue weakness cannot be measured.

To build the graph, we first create a string declaring the graph's nodes and edges. We then create a list of all observable variables, in this case, all variables in the graph except genetics, pre-game proprioception, intra-game proprioception and tissue weakness. We then pass all this information to the ```CausalGraph``` class, to create an instance of it.

In [ ]:
graph_str="""graph[directed 1 node[id "coach" label "coach"]
                        node[id "team motivation" label "team motivation"]
                        node[id "fitness" label "fitness"]
                        node[id "pre-game prop" label "pre-game prop"]
                        node[id "intra-game prop" label "intra-game prop"]                       
                        node[id "neuromusc fatigue" label "neuromusc fatigue"]
                        node[id "warm-up" label "warm-up"]
                        node[id "previous injury" label "previous injury"]
                        node[id "contact sport" label "contact sport"]
                        node[id "genetics" label "genetics"]
                        node[id "injury" label "injury"]
                        node[id "tissue disorder" label "tissue disorder"]
                        node[id "tissue weakness" label "tissue weakness"]
                        edge[source "coach" target "team motivation"]
                        edge[source "coach" target "fitness"]
                        edge[source "fitness" target "pre-game prop"]
                        edge[source "fitness" target "neuromusc fatigue"]
                        edge[source "team motivation" target "warm-up"]
                        edge[source "team motivation" target "previous injury"]
                        edge[source "pre-game prop" target "warm-up"]
                        edge[source "warm-up" target "intra-game prop"]
                        edge[source "contact sport" target "previous injury"]
                        edge[source "contact sport" target "intra-game prop"]
                        edge[source "intra-game prop" target "injury"]
                        edge[source "genetics" target "fitness"]
                        edge[source "genetics" target "neuromusc fatigue"]
                        edge[source "genetics" target "tissue disorder"]
                        edge[source "tissue disorder" target "neuromusc fatigue"]
                        edge[source "tissue disorder" target "tissue weakness"]
                        edge[source "neuromusc fatigue" target "intra-game prop"]
                        edge[source "neuromusc fatigue" target "injury"]
                        edge[source "tissue weakness" target "injury"]
                        ]
"""
observed_node_names=["coach", "team motivation", "fitness", "neuromusc fatigue",
                    "warm-up", "previous injury", "contact sport", "tissue disorder", "injury"]
treatment_name = "warm-up"
outcome_name = "injury"
G = CausalGraph(graph=graph_str, treatment_name=treatment_name, outcome_name=outcome_name,
                observed_node_names=observed_node_names)

We can easily create a plot of the graph using the ```view_graph``` method.

In [ ]:
G.view_graph()

The optimal backdoor set is a backdoor set comprised of observable variables that yields non-parametric
estimators of the interventional mean with the smallest asymptotic variance
among those that are based on observable backdoor sets. This optimal backdoor
set always exists when no variables are latent, and the algorithm is guaranteed to compute
it in this case. Under a non-parametric graphical model with latent variables,
such a backdoor set can fail to exist. 

The optimal minimal backdoor set is a minimal backdoor set comprised of observable variables that yields non-parametric
estimators of the interventional mean with the smallest asymptotic variance
among those that are based on observable minimal backdoor sets.

The optimal minimum cost backdoor set is a minimum cost backdoor set comprised of observable variables that yields non-parametric estimators of the interventional mean with the smallest asymptotic variance
among those that are based on observable minimum cost backdoor sets. The cost
of a backdoor set is defined as the sum of the costs of the variables that comprise it. Note that 
when all costs are equal, the optimal minimum cost backdoor set is the optimal backdoor set among those that 
have minimum cardinality.

These various optimal backdoor sets are not only optimal under
non-parametric graphical models and non-parametric estimators of interventional mean,
but also under linear graphical models and OLS estimators, per results in Henckel, Perkovic
and Maathuis (2020).

Next, we illustrate how to compute these backdoor sets for the example graph above, using the ```CausalIdentifier``` class. To compute the optimal backdoor set, optimal minimal backdoor set and optimal minimum cost backdoor set, we need to instantiate objects of the ```CausalIdentifier``` class, passing as ```method_name``` the values "efficient-adjustment", "efficient-minimal-adjustment" and "efficient-mincost-adjustment" respectively. Then, we need to call the ```identify_effect``` method, passing as an argument a list of conditional nodes, that is, the nodes that would be used to decide how to allocate treatment. As discussed above, in this example these nodes are previous injury and team motivation. For settings in which we are not interested in individualized interventions, we can just pass an empty list as conditional nodes.

In [ ]:
conditional_node_names=["previous injury", "team motivation"]

In [ ]:
ident_eff = CausalIdentifier(
        graph=G,
        estimand_type="nonparametric-ate",
        method_name="efficient-adjustment",
    )
print(ident_eff.identify_effect(conditional_node_names=conditional_node_names))

Thus, the optimal backdoor set is formed by previous injury, neuromusc fatigue, team motivation, tissue disorder and contact sport.

Similarly, we can compute the optimal minimal backdoor set.

In [ ]:
ident_minimal_eff = CausalIdentifier(
        graph=G,
        estimand_type="nonparametric-ate",
        method_name="efficient-minimal-adjustment",
    )
print(ident_minimal_eff.identify_effect(conditional_node_names=conditional_node_names))

Finally, we can compute the optimal minimum cost backdoor set. Since this graph does not have any costs associated with its nodes, we will not pass any costs to ```identify_effect```. The method will raise a warning, set the costs to one, and compute the optimal minimum cost backdoor set, which as stated above, in this case coincides with the optimal backdoor set of minimum cardinality.

In [ ]:
ident_mincost_eff = CausalIdentifier(
        graph=G,
        estimand_type="nonparametric-ate",
        method_name="efficient-mincost-adjustment",
    )
print(ident_mincost_eff.identify_effect(conditional_node_names=conditional_node_names))

Later, we will compute the optimal minimum cost backdoor set for a graph with costs associated with its nodes.

### An example in which sufficient conditions to guarantee the existance of an optimal backdoor set do not hold

For the graph below, it can be shown that the sufficient conditions to guarantee the existance of an optimal backdoor set introduced in [Smucler, Sapienza and Rotnitzky (2021)](https://doi.org/10.1093/biomet/asab018) do not hold. In this case, calling the ```identify_effect``` method of an instance of ```CausalIdentifier``` with attribute ```method_name``` equal to "efficient-adjustment" will raise an error. In fact, for this example it can be shown that an observable optimal backdoor set does not exist; see Example 3 in the aforementioned paper. However, optimal minimal and optimal minimum cost (cardinality) observable backdoor sets always exist, as long as there exists at least one backdoor set comprised of observable variables. For this graph, the optimal minimal and the optimal minimum cardinality backdoor sets are equal to the empty set.

In [ ]:
graph_str="""graph[directed 1 node[id "X" label "X"]
                        node[id "Y" label "Y"]
                        node[id "Z1" label "Z1"]
                        node[id "Z2" label "Z2"]
                        node[id "U" label "U"]                       
                        edge[source "X" target "Y"]
                        edge[source "Z1" target "X"]
                        edge[source "Z1" target "Z2"]
                        edge[source "U" target "Z2"]
                        edge[source "U" target "Y"]
                        ]
"""
observed_node_names = ['X', 'Y', 'Z1', 'Z2']
treatment_name = 'X'
outcome_name = 'Y'
G = CausalGraph(graph=graph_str, treatment_name=treatment_name, outcome_name=outcome_name,
                observed_node_names=observed_node_names)

In this example, the treatment intervention is static, thus there are no conditional nodes.

In [ ]:
ident_eff = CausalIdentifier(
        graph=G,
        estimand_type="nonparametric-ate",
        method_name="efficient-adjustment",
    )
results_eff=ident_eff.identify_effect()
results_eff

In [ ]:
ident_minimal_eff = CausalIdentifier(
        graph=G,
        estimand_type="nonparametric-ate",
        method_name="efficient-minimal-adjustment",
    )
print(ident_minimal_eff.identify_effect())

In [ ]:
ident_mincost_eff = CausalIdentifier(
        graph=G,
        estimand_type="nonparametric-ate",
        method_name="efficient-mincost-adjustment",
    )
print(ident_mincost_eff.identify_effect())

### An example in which there are no observable adjustment sets

In the graph below there are no adjustment sets comprised of only observable variables. In this setting, using any of the above methods will raise an error.

In [ ]:
graph_str="""graph[directed 1 node[id "X" label "X"]
                        node[id "Y" label "Y"]
                        node[id "U" label "U"]                       
                        edge[source "X" target "Y"]
                        edge[source "U" target "X"]
                        edge[source "U" target "Y"]
                        ]
"""
observed_node_names = ['X', 'Y']
treatment_name = 'X'
outcome_name = 'Y'
G = CausalGraph(graph=graph_str, treatment_name=treatment_name, outcome_name=outcome_name,
                observed_node_names=observed_node_names)

In [ ]:
ident_eff = CausalIdentifier(
        graph=G,
        estimand_type="nonparametric-ate",
        method_name="efficient-adjustment",
    )
results_eff=ident_eff.identify_effect()

### An example with costs

This is the graph in Figures 1 and 2 of [Smucler and Rotnitzky (2022)](https://arxiv.org/abs/2201.02037). Here we assume that there are positive costs associated to observable variables.

In [ ]:
graph_str="""graph[directed 1 node[id "L" label "L"]
                        node[id "X" label "X"]
                        node[id "K" label "K"]
                        node[id "B" label "B"]
                        node[id "Q" label "Q"]
                        node[id "R" label "R"]
                        node[id "T" label "T"]
                        node[id "M" label "M"]
                        node[id "Y" label "Y"]
                        node[id "U" label "U"]
                        node[id "F" label "F"]
                        edge[source "L" target "X"]
                        edge[source "X" target "M"]
                        edge[source "K" target "X"]
                        edge[source "B" target "K"]
                        edge[source "B" target "R"]
                        edge[source "Q" target "K"]
                        edge[source "Q" target "T"]
                        edge[source "R" target "Y"]
                        edge[source "T" target "Y"]
                        edge[source "M" target "Y"]
                        edge[source "U" target "Y"]
                        edge[source "U" target "F"]
                        ]
                        """
observed_node_names=["L", "X", "B", "K", "Q", "R", "M", "T", "Y", "F"]
conditional_node_names=["L"]
costs=[
    ("L", {"cost": 1}),
    ("B", {"cost": 1}),
    ("K", {"cost": 4}),
    ("Q", {"cost": 1}),
    ("R", {"cost": 2}),
    ("T", {"cost": 1}),
]
G = CausalGraph(graph=graph_str, treatment_name=treatment_name, outcome_name=outcome_name,
                observed_node_names=observed_node_names)

Notice how in this case we pass both the ```conditional_node_names``` list and the ```costs``` list to the ```identify_effect``` method.

In [ ]:
ident_mincost_eff = CausalIdentifier(
        graph=G,
        estimand_type="nonparametric-ate",
        method_name="efficient-mincost-adjustment",
    )
print(ident_mincost_eff.identify_effect(conditional_node_names=conditional_node_names, costs=costs))

We also compute the optimal minimal backdoor set, which in this case is different from the optimal minimum cost backdoor set.

In [ ]:
ident_minimal_eff = CausalIdentifier(
        graph=G,
        estimand_type="nonparametric-ate",
        method_name="efficient-minimal-adjustment",
    )
print(ident_minimal_eff.identify_effect(conditional_node_names=conditional_node_names))